This notebook's goal is to present a concept notebook of specviz, using workflows from the [IRAF splot tool](https://iraf.net/irafhelp.php?val=splot&help=Help+Page).  The feature/workflow list for splot came from a [trello board](https://trello.com/b/glMEUlE3/iraf-splot-feature-list), but the features are copied here to frame this notebook.

Now to guide the discussion, we create a glue viewer that resembles what the expected specviz main window should show. The actual application will have additional features like the plugin tray and data loading via the UI, but this shows the primary science-focused interaction space (the spectrum itself):

In [2]:
# THIS CELL IS FOR ILLUSTRATION OF WHAT THE UI LOOKS LIKE FOR EXPLAINING THE CONCEPTS, NOT INTENDED AS AN ACTUAL NOTEBOOK API FOR USERS.  IT PROBABLY NEEDS TO BE EXECUTED TO ACTUALLY SHOW ANYTHING THOUGH
import glue_jupyter
print('FYI: glupyter version:', glue_jupyter.__version__)
from glue_jupyter import jglue
from astropy.io import fits

app = jglue()
data = app.add_data(spectrum=spec)
viewer = app.profile1d(data='spectrum')
viewer

/home/erik/miniconda3/envs/jdaviz-dev2/lib/python3.8/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')


FYI: glupyter version: 0.2.dev143+g99ecd9e


Html(children=[Toolbar(children=[ToolbarItems(children=[BasicJupyterToolbar(children=[Tooltip(bottom=True, chi…

# Loading data/creating specviz:

Spectra to be loaded in specviz should be compatible with `Spectrum1D`, which should be viewed as the prime "input" to `specviz`.  So the default notebook-based loading operation would be:

In [ ]:
import specutils

spec_url = 'https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12'
spec = specutils.Spectrum1D.read(spec_url)

This spectrum should then be natively read by the specviz helper object. Note that this should automatically select the `.flux` part of the spectrum in the viewer, *not* require the user to do something to load that spectrum.

In [ ]:
import jdaviz

specviz = jdaviz.Specviz()
specviz.load_data(spec)

specviz  # <- could also be specviz.show(), but in the end it's just a wrapper in `Specviz._repr_html_`

To show what *should* pop up in the above cell, below I create a glue viewer that resembles what the expected specviz main window should show. The actual application will have additional features like the plugin tray and data loading via the UI, but this shows the primary science-focused interaction space (the spectrum itself):

In [9]:
# THIS CELL IS FOR ILLUSTRATION OF WHAT THE UI LOOKS LIKE FOR EXPLAINING THE CONCEPTS, NOT INTENDED AS AN ACTUAL NOTEBOOK API FOR USERS.  IT PROBABLY NEEDS TO BE EXECUTED TO ACTUALLY SHOW ANYTHING THOUGH
import glue_jupyter
print('FYI: glupyter version:', glue_jupyter.__version__)
from glue_jupyter import jglue
from astropy.io import fits

app = jglue()
data = app.add_data(spectrum=spec)
viewer = app.profile1d(data='spectrum')
viewer

FYI: glupyter version: 0.2.dev143+g99ecd9e


Html(children=[Toolbar(children=[ToolbarItems(children=[BasicJupyterToolbar(children=[Tooltip(bottom=True, chi…

While the above is sufficient for notebooks, in the desktop app there needs to be some way to load from a file path (the "open" button). This should use the exact same mechanism but use a file path or URL:

In [14]:
!curl https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12

In [17]:
ls

'Data interactions through CubeViz.ipynb'
'Example cubeviz spectral and spatial interactions.ipynb'
 Example_fitting.ipynb
'Mosviz concept notebook.ipynb'
 README.md
'Specviz from splot concept notebook.ipynb'


In [ ]:
specviz.load_data(spec_url)  #<- auto-downloads from the internet

# downalod to a local file and load it by name
!wget --content-disposition $spec_url
specviz.load_data('spec-1323-52797-0012.fits') 

Internally this should do the same as loading the `Spectrum1D` - i.e. first convert it to `Spectrum1D` using `read` and then load it into glue/jdaviz. That way the user can know exactly what is happening when they do it with a GUI button or at the command line.

## Uncertainties/mask

If the `Spectrum1D` object loaded contains an uncertainty and/or mask attribute, these should be loaded ointo glue when `load_data` is called.  However, by default only the spectrum itself should be *shown* at that time. The user can show it later using standard `jdaviz` interactions if desired. We *may* wish some syntactic convenience for the notebook as well - e.g.:

In [ ]:
specviz.show_uncertainty()
specviz.show_mask()

## SPLOT commands addressed by data loading operations 

### Options: images

*Description*:

The actual spectrum file that gets loaded when splot starts

### Query: next_image

*Description*:



### Query: new_image (for saving results of spectrum arithmetic)

*Description*:

 In response to  'i'  (write  current  spectrum)  this  parameter
        specifies  the  name  of a new image to create or existing image
        to overwrite.

Modifications  to  the spectra being analyzed may be saved using the
    'i' key in a new, the current, or other  existing  spectra.   A  new
    image  is  created  as  a new copy of the current spectrum and so if
    the  current  spectrum  is  part  of  a  multiple   spectrum   image 
    (including  a  long slit spectrum) the other spectra are copied.  If
    other spectra in the same image are then modified and saved use  the
    overwrite  option  to  replace then in the new output image.  If the
    output spectrum already exists then the overwrite flag must  be  set
    to  allow  modifying  the  data.   This  includes  the case when the
    output spectrum is the same as the input  spectrum.   The  only  odd
    case  here  is  when  the  input spectrum is one dimensional and the
    output spectrum is two  dimensional.   In  this  case  the  user  is
    queried for the line to be written.


# Zoom/Pan

The UI should support standard plot interaction options, including:

* Zooming *vertically* (without zooming horizontally)
* Zooming *horizontally* (without zooming vertically)
* A "reset to initial view" option - i.e., show the whole spectrum.
* Box-select zooming (implicitly also panning)
* Standard "hand"/touch-style panning
* Center view on click/keystroke (optional/lower priority)

### Explicily specifying limits

There should also be a way to manually set the plot limits:

* In the UI, which would use glue/bqplot's native controls for this
* In the notebook - e.g.:

In [ ]:
specviz.y_limits = (0, 1000) # <- uses whatever the current units are

# note this is already available as part of the glue viewer, so this should be syntactic sugar for something like:
specviz.spectrum_viewer.state.y_min = 0
specviz.spectrum_viewer.state.y_max = 1000

The notebook interface should also understand units.  That is, this should set the view to 5000 - 7000 angstroms regardless of what units the spectrum itself is in:

In [ ]:
from astropy import units as u

specviz.x_limits = (5000*u.angstrom, 7000*u.angstrom)

# if possible this should *also* defer to glue for unit-handling, but if that is non-trivial it could be part of the helper class

The above should *also* support `SpectralRegion`s as a convenience.  E.g., the above should be the same as:

In [ ]:
region = specutils.SpectralRegion(5000*u.angstrom, 7000*u.angstrom)
specviz.x_limits = region

Closely related, it should be possible to "flip" the x-axis.  With manual specification this should be as simple as:

In [ ]:
specviz.x_limits = (7000*u.angstrom, 5000*u.angstrom)

But it might also be useful to have a convenience:

In [ ]:
specviz.flip_x()

Which would also be in the UI as a checkbox (possibly best/already implemented in Glupyter?)

#### Autoscaling:

There should also be autoscale options which means "go from the beginning to end".  In the notebook, this might be as simple as:

In [ ]:
specviz.reset_view()

#which should be equivalent to:

specviz.x_limits = 'auto'
specviz.y_limits = 'auto'

# or similar

Additionally, to address the SPLOT `b` command use case, it should be possible to do:

In [ ]:
specviz.y_limits = (0, 'auto')

Which would mean "go from 0 to wherever the highest y value is in the spectrum"

Additionally, autoscaling might be better done as a method, because we may want more complicated ways of doing autoscaling:

In [ ]:
specviz.autoscale_y()  #<- same as above
specivz.autoscale_y(sigma_clipping=True)  #<- y-scale using sigma-clipping to get the range to view - useful when a spectrum has a few artifact outliers  

These notebook commands should eventually be exposed in the UI, probably via a plugin, although those using the glue machinery will already be available via glue itself.

## SPLOT commands addressed by pan/zoom:

### a -- Expand and autoscale to the data range between two cursor positions

*Description*:



### b -- Set the plot base level to zero rather than autoscaling

*Description*:



### c -- clear all windowing and redraw the current full spectrum.

*Description*:

Clear  all windowing and redraw the full current spectrum.  This
        redraws the spectrum and cancels any effects of  the  'a',  'z',
        and  'w'  keys.  The 'r' key is used to redraw the spectrum with
        the current windowing.
    


### r -- Redraw  the  spectrum with the current windowing.

*Description*:

Redraw  the  spectrum with the current windowing.  To redraw the
        full spectrum and cancel any windowing use the 'c' key.

### w -- window the graph

*Description*:

Window the graph.  For further help type '?'  to  the  "window:"
        prompt  or  see  help under gtools.  To cancel the windowing use
        'a'.

### z -- Zoom the graph by a factor of 2 in x.

*Description*:



### , -- Shift the graph window to the left.

*Description*:



### . -- Shift the graph window to the right.

*Description*:



### Options: wcreset

*Description*:

Option 
        "wreset"  resets  the  graph  limits  to  those specified by the
        xmin, xmax, ymin, ymax parameters whenever  a  new  spectrum  is
        plotted.  

### Options: flip

*Description*:

The  "flip" option selects that initially the spectra
        be plotted with decreasing  wavelengths. 

### Options: auto

*Description*:

Option  "auto"
        automatically  replots  the  graph  whenever  changes  are made.
        Otherwise the graph is replotted with  keystrokes  'c'  or  'r'.

### Options: zero

*Description*:

Option  "zero"  makes  the initial minimum y of the graphs occur
        at zero.  Otherwise the limits are set  automatically  from  the
        range  of  the  data  or  the  ymin  parameter. 

### Initial limits of graph (xmim,xmax,ymin,ymax)

*Description*:

 The  default  limits  for  the initial graph.  If INDEF then the
        limit is determined from the range of  the  data  (autoscaling).
        These  values  can  be changed interactively with 'w' window key
        options or the cursor commands ":/xwindow" and ":/ywindow"  (see
        gtools).


# Functionality for multidimensional spectra

SPLOT includes functionality (partly from broader IRAF) for doing spectral operations on multidimensional arrays like "2d" spectra or data cubes.  In general this is not a target for `specviz` so it is not a priority of the UI, but much of the SPLOT workflow can be addressed from the notebook by slicing spectrum objects:

In [25]:
import numpy as np
spectrum2d = np.random.randn(50, 1024) # A fake "2d spectrum" to approximate a 50-pixel slit with 1024 spectral pixels along columns

spec = specutils.Spectrum1D(spectral_axis=np.arange(1024)*u.pixel,
                            flux=spectrum2d[25]*u.adu)
specviz.load_data(spec)

Or alternatively, if a data cube or other construct can be loaded as a `SpectrumCollection`, it could simply be loaded as:

In [ ]:
speccoll = specutils.SpectrumCollection.read('...') # assume this is a 50x50xn_spec data cube

specviz.load_data(speccoll[20, 30])  # shows the 20, 30th spaxel

The above could potentially be exposed as part of a custom file-loading UI for the desktop tool, but it is not clear that this is a priority so should probably be considered a "nice to have" or "stretch goal".

SPLOT also contains some functionality related to the above focused on going through several spectra in sequence (generally tied with multi-object spectrograph results) - this use cases is the target of mosviz so is explicitly not a goal here.

## SPLOT commands addressed by multidimensional loading or mosviz

### Options: line and band: 

*Description*:

One  may  use  an         image  section to select a desired column, line, or band

**line, band**
        The  image  line/aperture  and  band  to  plot  in  two or three
        dimensional images.   For  multiaperture  spectra  the  aperture
        specified  by  the  line  parameter  is  first sought and if not
        found the specified image  line  is  selected.   For  other  two
        dimensional   images,  such  as  long  slit  spectra,  the  line 
        parameter specifies a line or column.  Note  that  if  the  line
        and  band  parameters  are specified on the command line it will
        not be possible to change them interactively.


### ( -- In multiaperture spectra go to the  spectrum  in  the  preceding         image  line.

*Description*:

In multiaperture spectra go to the  spectrum  in  the  preceding
        image  line.   If  there  is only one line go to the spectrum in
        the preceding band.

### ) -- In multiaperture spectra go to the  spectrum  in  the  following         image  line.

*Description*:

In multiaperture spectra go to the  spectrum  in  the  following
        image  line.   If  there  is only one line go to the spectrum in
        the following band.

### q -- quit and go to the next input spectrum

*Description*:

Quit and go on to next input spectrum.  After the last  spectrum
        exit.

# Miscellaneous SPLOT functionality

### Options: histogram

*Description*:

Option "histogram"  plots  the  spectra  in  a  histogram  style
        rather  than  connecting  the  pixel  centers. 

`specviz` should default to histogram style, and it's a "nice to have" option to be able to toggle.  But either way this is non-critical.

### Options: save_file

*Description*:

save_file
        The  file  to  contain  any  results generated by the equivalent
        width or deblending functions.  Results are added to  this  file
        until  the  file is deleted.  If the filename is null (""), then
        no results are saved.


`specviz` will meet this need in two ways:

1. For notebook-oriented workflows, the notebook itself is the "save file" - e.g., analysis should
2. For desktop tool workflows, a log should be available to track operations. This could/should use the standard Python logging functionality, but to reproduce the specific needs of SPLOT, plugins should have a way to explicitly say "log my results". This log file should be accessible in the UI which would then have a "save" option for the user to record this if they want to.

### Labels: various forms of labeling

*Description*:

    Labels:

        :label <label> <format>
            Add a label at the cursor position.
        
        :mabove <label> <format>
            Add  a  tick mark and label above the spectrum at the cursor
            position.
        
        :mbelow <label> <format>
            Add a tick mark and label below the spectrum at  the  cursor
            position.
        
        The  label  must  be  quoted  if  it  contains  blanks.  A label
        beginning with % (i.e. %.2f) is treated as a format  for  the  x
        cursor  position.   The  optional  format is a gtext string (see
        help on "cursors").   The  labels  are  not  remembered  between
        redraws.

`bqplot` and `glue`/`glupyter` have mechanisms for setting labels.  It might be a bonus to add convenient accessors to make setting such labels easier along the lines of the cell below (but not this is not critical functionality):

In [ ]:
specviz.set_flux_label('My Nobel Prize-winning spectrum', include_unit=False)
specviz.set_spec_label('My special name for the spectral axis', include_unit=True)

# Specviz functionality in mosviz or cubeviz

Note that some of the functionality here is also desirable in cubeviz or mosviz since both have a "specviz view". While the plugin architecture means plugins built for specviz can also work in other `jdaviz` configurations (as long as the viewers are named appropriately), the same does not hold for the notebook interactions outlined above.  For those, however, the below idiom should be used:

In [ ]:
cubeviz = jdaviz.Cubeviz(...)
... do whatever is necessary to load the relevant cubeviz data ...

specviz = jdaviz.Specviz(app=cubeviz.app)
specviz.autoscale_y() #<- assuming the viewer in cubeviz is the same name, this should work transparently

# SPLOT functionality NOT YET CATEGORIZED

### Fitting: function

*Description*:

Function   to   be  fit  to  the  spectra.   The  functions  are 
        "legendre"   (legendre   polynomial),   "chebyshev"   (chebyshev  
        polynomial),  "spline1"  (linear  spline),  and "spline3" (cubic
        spline).  The functions may be abbreviated.


### Fitting: order

*Description*:

**order**
        The order of the polynomials or the number of spline pieces.

### Fitting: low_reject, high_reject

*Description*:

Rejection limits below  and  above  the  fit  in  units  of  the
        residual  sigma.   Unequal  limits  are  used to reject spectral
        lines on one side of the continuum during continuum fitting.


### Fitting: niterate

*Description*:

Number of rejection iterations.

### Fitting: grow

*Description*:

When a pixel is rejected, pixels within  this  distance  of  the
        rejected pixel are also rejected.


### Fitting: markrej

*Description*:

Mark  rejected  points?   If  there  are many rejected points it
        might be desired to not mark rejected points.

### Query: overwrite

*Description*:



### Query: Spec2

*Description*:



### Query: constant

*Description*:



### Query: wavelength

*Description*:



### Query: linelist

*Description*:



### Query: wstart, wend, dw

*Description*:



### Query: boxsize (for smoothing)

*Description*:



### SPACE -- prints  the  cursor  position  and value of the nearest pixel.

*Description*:



### d -- Mark two  continuum  points  and  fit  (deblend)  multiple  line         profiles.

*Description*:

Detailed description in the splot help file...


### e -- Measure equivalent width

*Description*:

Measure equivalent width by marking two continuum points  around
        the  line  to  be  measured.  The linear continuum is subtracted
        and the flux is determined by simply  summing  the  pixels  with
        partial  pixels  at  the  ends.   Returned  values  are the line
        center, continuum at the region center, flux above or below  the
        continuum, and the equivalent width.

### f -- Arithmetic function mode

*Description*:


Enter  arithmetic  function  mode.  This  mode allows arithmetic
        functions to be applied to the spectrum. The  pixel  values  are
        modified  according  to the function request and may be saved as
        a new spectrum with the 'i' command.  Operations with  a  second
        spectrum  are  done  in wavelength space and the second spectrum
        is automatically resampled if necessary.   If  one  spectrum  is
        longer  than  the  other,  only the smaller number of pixels are
        affected.  To exit this mode type 'q'.
        
The keystrokes listed in the checklist are available  in  the  function  mode.
        Binary  operations  with a constant or a second spectrum produce
        a query for the constant value or spectrum name.
        

### g -- get another spectrum

*Description*:

The current spectrum is replaced by the
        new  spectrum.   The  aperture/line  and  band  are  queried  is 
        necessary.


### h -- Measure  equivalent  widths assuming a gaussian profile with the         width measured at a specified point.

*Description*:

Measure  equivalent  widths assuming a gaussian profile with the
        width measured at a specified point.  Note that this  is  not  a
        gaussian  fit (see 'k' to fit a gaussian)!  The gaussian profile
        determined here may be subtracted with the '-'  key.   A  second
        cursor key is requested with one of the following values:

        a   Mark  the  continuum  level  at  the line center and use the
            LEFT half width at the half flux point.
        
        b   Mark the continuum level at the  line  center  and  use  the
            RIGHT half width at the half flux point.
        
        c   Mark  the  continuum  level  at  the line center and use the
            FULL width at the half flux point.
        
        l   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use  the LEFT width at that flux
            point.
        
        r   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use the RIGHT width at that flux
            point.
        
        k   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use  the FULL width at that flux
            point.

### i --  Write the current spectrum out to a new or existing image.

*Description*:



### j -- Set  the  value  of  the  nearest pixel to the x cursor to the y         cursor position.

*Description*:



### k -- Mark two continuum points and fit a single  line  profile.

*Description*:

Mark two continuum points and fit a single  line  profile.   The
        second  key  selects  the type of profile: g for gaussian, l for
        lorentzian, and v for voigt.  Any other second key  defaults  to
        gaussian.   The center, continuum at the center, core intensity,
        integrated flux, equivalent width, and  FWHMs  are  printed  and
        saved  in  the  log file.  See 'd' for fitting multiple profile
        and '-' to subtract the fit.

### l -- convert to f-lambda

*Description*:

Convert to flux per unit wavelength (f-lambda). The spectrum  is
        assumed  to  be  flux  calibrated  in  flux  per  unit frequency
        (f-nu).  See also 'n'.

### m -- Compute the mean, RMS, and signal-to-noise over a region  marked         with two x cursor positions.

*Description*:



### n -- convert to fnu

*Description*:



### o -- set overplot flag

*Description*:

Set  overplot  flag.   The  next  plot will overplot the current
        plot.  Normally this key is immediately followed by one of  'g',
        '#',  '%',  '(',  or  ')'.   The  ":overplot"  colon command and
        overplot parameter option may be used to set overplotting to  be
        permanently on.

### p -- define a linear wavelength scale

*Description*:

Define  a  linear  wavelength  scale.  The user is queried for a
        starting  wavelength  and  an  ending  wavelength.   If   either 
        (though  not  both)  are  specified  as  INDEF  a  dispersion is
        queried for and used  to  compute  an  endpoint.   A  wavelength
        scale  set this way will be used for other spectra which are not
        dispersion corrected.
    

### s -- smooth via boxcar.

*Description*:

Smooth via a boxcar.  The user is prompted for the box size.

### t -- fit a function to the spectrum.

*Description*:


Fit a function to the spectrum using the ICFIT mode.   Typically interactive  rejection is used to exclude spectra lines from the fit in order to fit a  smooth  continuum.   A  second  keystroke selects what to do with the fit.
        
        /   Normalize  by  the  fit.   When  fitting  the continuum this
            continuum normalizes the spectrum.
        
        -  Subtract the fit.  When fitting the continuum this  continuum
           subtracts the spectrum.
        
        f   Replace the spectrum by the fit.
        
        c   Clean  the  spectrum by replacing any rejected points by the
            fit.
  
        n   Do the fitting but leave the spectrum unchanged  (a  NOP  on
            the  spectrum).   This  is  useful to play with the spectrum
            using the capabilities of ICFIT.
        
        q   Quit  and  don't  do  any  fitting.   The  spectrum  is  not 
            modified.


### u -- Adjust  the user coordinate scale.

*Description*:

Adjust  the user coordinate scale.  There are three options, 'd' mark a position with the  cursor  and  doppler  shift  it  to  a specified  value,  'z'  mark  a  position  with  the  cursor and zeropoint shift it  to  a  specified  value,  or  'l'  mark  two postions   and   enter   two  values  to  define  a  linear  (in  wavelength) dispersion scale.  The  units  used  for  input  are those  currently  displayed.   A  wavelength  scale set this way will  be  used  for  other  spectra  which  are  not  dispersion  corrected.

### v -- Toggle  to  a velocity scale using the position of the cursor as         the velocity origin and back.

*Description*:



### # -- Get  a  different  line  in   multiaperture   spectra   or   two          dimensional images.

*Description*:

Get  a  different  line  in   multiaperture   spectra   or   two 
        dimensional images.  The aperture/line/column is queried.

### % -- Get a different band in a three dimensional image.

*Description*:



### $ -- Switch    between   physical   pixel   coordinates   and   world           (dispersion) coordinates.

*Description*:



### - -- Subtract the fit

*Description*:

Subtract the fits generated by the 'd'  (deblend),  'k'  (single
        profile  fit),  and  'h'  (gaussian  of  specified  width).  The
        region to be subtracted is marked with two cursor positions.

### I -- Force a fatal error interupt to leave the graph.

*Description*:

Force a fatal error interupt to leave the graph.  This  is  used
        because  the  normal  interupt  character is ignored in graphics
        mode.

### :show -- Page the full output of  the  previous  deblend  and  equivalent         width measurements.

*Description*:



### : log, nolog -- Enable/Disable  logging  of  measurements  to  the file specified by the parameter save_file.

*Description*:



### :dispaxis <val> -- Show or change dispersion axis for 2D images.

*Description*:



### :sum -- Show or change summing for 2D images.

*Description*:



### :units -- Change the coordinate units in the plot.

*Description*:



### :# -- Add comment to logfile

*Description*:



### Units

*Description*:


        Dispersion coordinate units for the plot.  If the  spectra  have
        known  units,  currently  this is generally Angstroms, the units
        may be converted to other units for  plotting  as  specified  by
        this  task  parameter.  If this parameter is the null string and
        the  world  coordinate  system  attribute   "units_display"   is 
        defined  then  that  will be used.  If both this task parameters
        and "units_display" are not given then the  spectrum  dispersion
        units   will   be   used.    The   units  may  also  be  changed 
        interactively.  See the units section of the package help for  a
        further description and available units.


# Won't-address SPLOT features/options

Notes indicate why this are "won't-address" - in most cases it's just "not relevant"

### x -- "Etch-a-sketch"  mode.  Straight   lines   are   drawn   between          successive  positions  of the cursor.

*Description*:

"Etch-a-sketch"  mode.  Straight   lines   are   drawn   between 
        successive  positions  of the cursor. Requires 2 cursor settings
        in x.  The nearest pixels are used as the endpoints.  To draw  a
        line  between  arbitrary  y  values  first use 'j' to adjust the
        endpoints or set the "xydraw" option.
        
**NOTES** It's not clear any science use case really needs this, and if desired it is better to be done using plotting machinery directly.

### Options: xydraw

*Description*:

Option "xydraw"
        changes the 'x' draw key to use both x and y cursor  values  for
        drawing  rather  than  the  nearest pixel value for the y value.
        
**NOTES** (see `x` above)

### Options: nosysid

*Description*:

Option "nosysid"
        excludes  the  system  banner  from  the  graph  title.
        
**NOTES**: Not relevant for jdaviz

### Options: overplot

*Description*:

The  "overplot"  options 
        overplots all graphs and a screen erase  only  occurs  with  the
        redraw key.

**NOTES**: Natively supported by glue's layer functionality 

### Options: nerrsample (number of samples for error calculation)

*Description*:

nerrsample = 0
        Number of samples for the error computation.  A value less  than
        10  turns  off  the  error  computation.   A value of ~10 does a
        rough error analysis, a value of ~50  does  a  reasonable  error
        analysis,  and a value >100 does a detailed error analysis.  The
        larger this value the longer the analysis takes.
        
        
**NOTES** Not relevant for specviz because uncertainty handling is part of the fitting/etc process not a global property of the viz tool

### Options: sigma0 (used for calculating uncertainties)

*Description*:

** sigma0 = INDEF, invgain = INDEF **
The pixel sigmas are modeled by the formula:
        
            sigma**2 = sigma0**2 + invgain * I
        
where I is the pixel value and "**2" means  the  square  of  the
        quantity.   If  either parameter is specified as INDEF or with a
        value less than zero then no sigma estimates are made and so  no
        error estimates for the measured parameters are made.
        
**NOTES** Not relevant for specviz because uncertainty handling is part of the fitting/etc process not a global property of the viz tool

### ? -- Page help information

*Description*:

**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used

### / -- Cycle through short status line help

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### :/help -- get help on GTOOLS options

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### :.help -- get help on standard cursor mode options

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### Calibration: star_name

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: mag

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: magband

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: teff

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: caldir

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: fnuzero

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### y -- Overplot standard star values from a calibration file.

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

# Temporary space 

In [3]:
ss = app.data_collection.subset_groups[0].subsets[0].subset_state
ss.lo, ss.hi

IndexError: tuple index out of range

### Code to import notebook content from Trello

In [ ]:
import requests

j = requests.get('https://trello.com/b/glMEUlE3.json').json()

In [ ]:
import nbformat as nbf

nb = nbf.read('Specviz from splot concept notebook.ipynb', nbf.NO_CONVERT)
nbfapi = getattr(nbf, 'v'+str(nb['nbformat']))

In [ ]:
for card in j['cards']:
    if card['name'] == 'IRAF Splot help text':
        continue
    markdown_text = '### '+ card['name'] + '\n\n*Description*:\n\n' + card['desc']
    nb['cells'].append(nbfapi.new_markdown_cell(markdown_text))
nbf.write(nb, 'output.ipynb')
# now manually move that to overwrite this file

In [42]:
splot_help_file = requests.get(j['cards'][0]['attachments'][0]['url'])
print(splot_help_file.text)

SPLOT (Jul95)                 noao.onedspec                SPLOT (Jul95)



NAME
    splot -- plot and analyze spectra
    
    
USAGE
    splot images [line [band]]
    
    
PARAMETERS
    
    images
        List of images (spectra) to plot.  If the image is 2D or 3D  the
        line  and  band  parameters  are  used.   Successive  images are
        plotted following each 'q'  cursor  command.   One  may  use  an
        image  section to select a desired column, line, or band but the
        full image will be in memory and any  updates  to  the  spectrum
        will be part of the full image.
    
    line, band
        The  image  line/aperture  and  band  to  plot  in  two or three
        dimensional images.   For  multiaperture  spectra  the  aperture
        specified  by  the  line  parameter  is  first sought and if not
        found the specified image  line  is  selected.   For  other  two
        dimensional   images,  such  as  long  slit  spectra,  the  line 
      